# Parameters List
1. k_batch=list(np.arange(1.0, 4.0, 0.5)) # k parameter

2. k_instance=list(np.arange(1.0, 4.0, 0.5)) # This can be set the same as k_batch and hence excluded

3. count_threshold_measure=['median','max'] # alpha parameter

4. logical_parameter=['and','or']



In [1]:
import numpy as np
k_batch=list(np.arange(0.5, 4.5, 0.25))
count_threshold_measure=['median','max']
logical_parameter=['and','or']

In [2]:
import itertools

hyperparameter_space = {
    'k_batch': k_batch,
    'count_threshold_measure':count_threshold_measure,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))


In [3]:
all_combinations

[(0.5, 'median', 'and'),
 (0.5, 'median', 'or'),
 (0.5, 'max', 'and'),
 (0.5, 'max', 'or'),
 (0.75, 'median', 'and'),
 (0.75, 'median', 'or'),
 (0.75, 'max', 'and'),
 (0.75, 'max', 'or'),
 (1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.25, 'median', 'and'),
 (1.25, 'median', 'or'),
 (1.25, 'max', 'and'),
 (1.25, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (1.75, 'median', 'and'),
 (1.75, 'median', 'or'),
 (1.75, 'max', 'and'),
 (1.75, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.25, 'median', 'and'),
 (2.25, 'median', 'or'),
 (2.25, 'max', 'and'),
 (2.25, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (2.75, 'median', 'and'),
 (2.75, 'median', 'or'),
 (2.75, 'max', 'and'),
 (2.75, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max

In [4]:
len(all_combinations)

64

### Design the Process flow for grid search

1. Autoencoder is pre-trained , which will be used for all combinations ,this will reduce the processing overhead
2. The validation data needs to be processed once 
3. The iterations will start from threshold computations 
4. The objective is least warnings and detections on normal data set , least detection delay , and more warnnings after detection point
5. We can set the normal data size 20 batches , drifted data size also 20 batches , both combined with drift starting from batch 20
6. We can assign weights to factors in (5) and define a score



In [5]:
total_batches=40
batch_size=32
drift_point=20

WnDs=[i for i in range(0,21)]
print("Warnings in Drift Stream before drift point",WnDs)
DnDs=[i for i in range(0,21)]
print("Detections in Drift Stream before drift point",DnDs)
delay=[ i for i in range(21)]
print("Detection Delay:",delay)
DdDs=[i for i in range(19)]
print("Detections after drift point:",DdDs)
WdDs=[i for i in range(19)]
print("Warnings after drift point:",WdDs)

Warnings in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detection Delay: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Warnings after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [6]:
WnDS=[i for i in range(0,21)]
DnDs=[i for i in range(0,21)]
delay=[ i for i in range(21)]
DdDs=[i for i in range(19)]
WdDs=[i for i in range(19)]

In [7]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    best_value = float('-inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value > best_value:
                            best_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, best_value

# Find the best parameters and the corresponding value
best_parameters, best_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Best parameters:', best_parameters)
print('Value:', best_value)

Best parameters: (0, 0, 0, 18, 18)
Value: 54


In [8]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    worst_value = float('inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value < worst_value:
                            worst_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, worst_value

# Find the best parameters and the corresponding value
best_parameters, worst_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Worst parameters:', best_parameters)
print('Value:', worst_value)

Worst parameters: (20, 20, 20, 0, 0)
Value: -120


# Dataset 

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing


from skmultiflow.data import HyperplaneGenerator
from scipy.stats import norm
import random
from scipy import stats 


import tensorflow
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import anderson

In [10]:
data=pd.read_csv('D:/Concept Drift Papers for Proposal Defence/Datasets/NOAA.csv')

In [11]:
data

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,class
0,19.8,14.0,1019.6,8.4,9.9,15.9,28.9,14.0,1
1,26.8,22.2,1006.2,8.1,10.9,19.0,34.0,21.0,1
2,34.6,32.9,1004.6,3.9,13.8,22.0,36.0,33.1,2
3,26.4,21.5,1006.9,8.0,18.7,30.1,39.9,16.0,2
4,14.7,7.9,1009.9,8.1,14.1,22.0,21.0,9.0,1
...,...,...,...,...,...,...,...,...,...
18154,32.6,21.9,1022.7,7.0,12.4,26.0,41.0,24.8,2
18155,36.8,25.4,1014.9,7.0,10.0,19.0,57.2,21.2,1
18156,41.9,29.9,1010.0,7.0,4.7,9.9,62.6,28.4,1
18157,42.4,29.7,1011.3,7.0,3.3,8.9,51.8,32.0,1


In [12]:
#drifted_data=pd.read_csv('stream_top25.csv')

In [13]:
#del drifted_data['class']

In [14]:
#drifted_data=drifted_data[0:1280]

In [15]:
#drifted_data = drifted_data.drop(['Unnamed: 0'], axis=1) 

### Add Noise

In [16]:
def add_noise(df,std_dev=0.03):
    df_noisy=df.copy()
    for col in df.select_dtypes(include=['float64','int64']).columns:
        noise=np.random.normal(0,std_dev,size=len(df))
        df_noisy[col] = df_noisy[col] + noise
    return df_noisy


In [17]:
#drifted_data = add_noise(drifted_data, std_dev=0.03) 

## 3. Data Preprocessing 
1. Initial 70% data is used for training ( training and validation for AE)
2. Next 20 % data is used as a validation set to compute thresholds
3. Next 10 % data is used as a test stream initilally without drift 
  

In [18]:
def normalize_encode_split(data,label_col,pos_val,neg_val):        
    # Apply Label Encoding
    
    for col in data.columns:
        if ((data[col].dtype=='object')and (col!=label_col)): 
            data = pd.get_dummies(data, columns=[col])
            
   # Apply Minmax Normalization 
    for col in data.columns:
        if (((data[col].dtype=='float64')or(data[col].dtype=='int64')) and (col!=label_col)): 
            data[col] = np.round((data[col] - data[col].min()) / (data[col].max() - data[col].min()) ,3)  
            
            
    # Split into training  , test (validation set 1) and stream (Validation Set II)
            
    train=data[0:int(len(data)*0.70)]
    test=data[int((0.70*len(data))):int((0.90*len(data)))]
    stream=data[int((0.90*len(data))):len(data)]
    
    train_positives = train[train[label_col] == pos_val]
    train_negatives = train[train[label_col] == neg_val]
    
    X_positive=train_positives.drop([label_col],axis=1)
    X_negative=train_negatives.drop([label_col],axis=1)
    return train, test, X_positive,X_negative , stream      

In [19]:
train, test, X_positive,X_negative , stream =normalize_encode_split(data,'class',2,1)

In [20]:
#train, test, X_positive,X_negative , stream, y_positive ,y_negative =normalize_encode_split(data,'output',1,0)

In [21]:
drifted_data=stream.copy()
del drifted_data['class']
#drifted_data=drifted_data[0:1280]
drifted_data = add_noise(drifted_data, std_dev=0.05) 

### Best Autoencoder Hyperparameters will be deternmined for each data set separetely 

In [22]:
stream

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,class
16343,0.352,0.472,0.008,0.137,0.222,0.191,0.338,0.399,1
16344,0.444,0.566,0.006,0.012,0.168,0.130,0.397,0.504,2
16345,0.466,0.572,0.006,0.137,0.158,0.210,0.448,0.515,2
16346,0.462,0.556,0.007,0.226,0.208,0.210,0.440,0.523,1
16347,0.376,0.477,0.008,0.210,0.201,0.170,0.347,0.429,1
...,...,...,...,...,...,...,...,...,...
18154,0.425,0.461,0.009,0.278,0.441,0.512,0.423,0.446,2
18155,0.465,0.495,0.008,0.278,0.355,0.363,0.561,0.412,1
18156,0.514,0.538,0.007,0.278,0.165,0.170,0.606,0.480,1
18157,0.519,0.536,0.007,0.278,0.115,0.149,0.515,0.515,1


In [23]:
def autoencoder (train,epochs,val_set):
    
    # set bottleneck 1/3 of the input layer size
    #encoding_dim = int(np.floor(train.shape[-1]/3))
       
    # input layer size= # of attributes in the dataset after one-hot encoding
    input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
    encoded = layers.Dense(6, activation='relu')(input_layer) # Code Layer 1
    encoded=layers.Dense(2,activation='relu')(encoded) # Bottleneck
    decoded=layers.Dense(6,activation='relu')(encoded)# Decode Layer 1
    decoded = layers.Dense(train.shape[1], activation='sigmoid')(decoded) # Output Layer
          
    autoencoder = tensorflow.keras.Model(input_layer, decoded)
    # above model maps an input to its reconstruction 
   
    autoencoder.compile(optimizer='adam', loss='mse') # Train autoencoder 
    history=autoencoder.fit(train,train, 
                epochs=epochs,
                batch_size=32,
                shuffle=True,
                validation_data=(val_set, val_set)).history
    return  autoencoder , history

In [24]:
def train_encoders(X_Positive,X_Negative, epochs):
    
    X_Positive_train=X_Positive[0:int(len(X_Positive)*0.90)]
    X_Positive_test=X_Positive[int((0.90*len(X_Positive))):len(X_Positive)-1]
    
    X_Negative_train=X_Negative[0:int(len(X_Negative)*0.90)]
    X_Negative_test=X_Negative[int((0.90*len(X_Negative))):len(X_Negative)-1]

    print("Training Autoencoder on Positive Examples ")
    encoder_pos_class, history_positive_class=autoencoder(X_Positive_train,epochs,X_Positive_test)
    print("Training Autoencoder on Negative Examples ")
    encoder_neg_class,history_negative_class=autoencoder(X_Negative_train,epochs,X_Negative_test)
    
    return encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class

In [25]:
encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class=train_encoders(X_positive,X_negative, 100)

Training Autoencoder on Positive Examples 
Epoch 1/100
106/106 [==============================] - 2s 15ms/step - loss: 0.0741 - val_loss: 0.0642
Epoch 2/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0517 - val_loss: 0.0221
Epoch 3/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0197 - val_loss: 0.0166
Epoch 4/100
106/106 [==============================] - ETA: 0s - loss: 0.015 - 0s 3ms/step - loss: 0.0157 - val_loss: 0.0152
Epoch 5/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0146 - val_loss: 0.0140
Epoch 6/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0134 - val_loss: 0.0132
Epoch 7/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0123 - val_loss: 0.0125
Epoch 8/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0119 - val_loss: 0.0120
Epoch 9/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0113 - val_loss: 0.0117
Epoch 10/100
106/106 [=

106/106 [==============================] - 0s 3ms/step - loss: 0.0033 - val_loss: 0.0048
Epoch 81/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0033 - val_loss: 0.0049
Epoch 82/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0047
Epoch 83/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0033 - val_loss: 0.0048
Epoch 84/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0034 - val_loss: 0.0049
Epoch 85/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0048
Epoch 86/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0048
Epoch 87/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0049
Epoch 88/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0032 - val_loss: 0.0048
Epoch 89/100
106/106 [==============================] - 0s 3ms/step - loss: 0.0034 - val_loss: 

Epoch 60/100
252/252 [==============================] - 1s 3ms/step - loss: 0.0072 - val_loss: 0.0082
Epoch 61/100
252/252 [==============================] - 1s 3ms/step - loss: 0.0072 - val_loss: 0.0085
Epoch 62/100
252/252 [==============================] - 1s 3ms/step - loss: 0.0073 - val_loss: 0.0082
Epoch 63/100
252/252 [==============================] - 1s 3ms/step - loss: 0.0073 - val_loss: 0.0081
Epoch 64/100
252/252 [==============================] - 1s 3ms/step - loss: 0.0074 - val_loss: 0.0085
Epoch 65/100
252/252 [==============================] - 1s 2ms/step - loss: 0.0070 - val_loss: 0.0085
Epoch 66/100
252/252 [==============================] - 1s 2ms/step - loss: 0.0072 - val_loss: 0.0083
Epoch 67/100
252/252 [==============================] - 1s 3ms/step - loss: 0.0074 - val_loss: 0.0083
Epoch 68/100
252/252 [==============================] - 1s 2ms/step - loss: 0.0073 - val_loss: 0.0085
Epoch 69/100
252/252 [==============================] - 1s 2ms/step - loss: 0.0073

In [26]:
test

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,class
12711,0.011,0.034,0.012,0.681,0.323,0.340,0.026,0.039,1
12712,0.040,0.058,0.012,0.468,0.344,0.278,0.034,0.107,1
12713,0.001,0.029,0.015,0.266,0.613,0.507,0.000,0.077,1
12714,0.041,0.051,0.013,0.669,0.319,0.318,0.093,0.088,1
12715,0.144,0.178,0.010,0.774,0.086,0.083,0.196,0.173,1
...,...,...,...,...,...,...,...,...,...
16338,0.176,0.266,0.010,0.383,0.211,0.340,0.312,0.172,1
16339,0.346,0.449,0.006,0.077,0.290,0.380,0.312,0.371,1
16340,0.169,0.261,0.009,0.415,0.222,0.340,0.219,0.151,1
16341,0.267,0.336,0.009,0.512,0.082,0.108,0.252,0.296,1


## Validation Data

In [27]:
def make_batches(test_data): 
        data=np.array(test_data)
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches

In [28]:
def process_validation_data(test,encoder_pos_class,encoder_neg_class,batch_size=32):
   
    # From the test data ,separate positive and negative class data and remove the class column

    test_pos_class=test[test['class']==2]
    test_neg_class=test[test['class']==1]
    del test_pos_class['class']
    del test_neg_class['class']
       
       
    # This function make batches of the data stream
     
    def make_batches(test_data):
        data=np.array(test_data)
        batch_size=32
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches
    #
    
    batches_pos=make_batches(test_pos_class)
    batches_neg=make_batches(test_neg_class)
    
    # This function computes reconconstruction error for each instance as well as average reco.error of a batch passed to it. 
    def compute_instance_loss_batch_loss(batch,batch_size,encoder):
        mse_list=[]
        mse_sum=0  
        for i in range(0,batch.shape[0]):
            ROW = np.array([batch[i]])
            pred= encoder.predict(ROW) 
            mse = np.mean(np.power(batch[i] - pred, 2))
            mse_list.append(mse)
            mse_sum+=mse
        avg_mse=mse_sum/batch_size
        return mse_list,avg_mse   

            
    # This function computes recon.errr of all the batches .  
    def process_all_batches(batches,encoder):
        batch_avg_mse=[]
        batch_mse_values={}
        for b in batches:
            print(b,end=" ")
            mse_list,average_mse=compute_instance_loss_batch_loss(batches[b],batch_size,encoder)
            batch_avg_mse.append(average_mse)
            batch_mse_values[b]=mse_list
        return batch_avg_mse,batch_mse_values

    batch_avg_mse_pos,batch_mse_values_pos=process_all_batches(batches_pos,encoder_pos_class)
    batch_avg_mse_neg,batch_mse_values_neg=process_all_batches(batches_neg,encoder_neg_class)
       
    
    
    return  batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg
    
    
    
   
    

In [29]:
batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg=process_validation_data(test,encoder_pos_class,encoder_neg_class)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 

In [30]:
k_parameter=list(np.arange(0.5, 4.5, 0.25))
alpha_parameter=['median','max']
logical_parameter=['and','or']
import itertools

hyperparameter_space = {
    'k': k_parameter,
    'count_threshold_measure':alpha_parameter,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))
all_combinations

[(0.5, 'median', 'and'),
 (0.5, 'median', 'or'),
 (0.5, 'max', 'and'),
 (0.5, 'max', 'or'),
 (0.75, 'median', 'and'),
 (0.75, 'median', 'or'),
 (0.75, 'max', 'and'),
 (0.75, 'max', 'or'),
 (1.0, 'median', 'and'),
 (1.0, 'median', 'or'),
 (1.0, 'max', 'and'),
 (1.0, 'max', 'or'),
 (1.25, 'median', 'and'),
 (1.25, 'median', 'or'),
 (1.25, 'max', 'and'),
 (1.25, 'max', 'or'),
 (1.5, 'median', 'and'),
 (1.5, 'median', 'or'),
 (1.5, 'max', 'and'),
 (1.5, 'max', 'or'),
 (1.75, 'median', 'and'),
 (1.75, 'median', 'or'),
 (1.75, 'max', 'and'),
 (1.75, 'max', 'or'),
 (2.0, 'median', 'and'),
 (2.0, 'median', 'or'),
 (2.0, 'max', 'and'),
 (2.0, 'max', 'or'),
 (2.25, 'median', 'and'),
 (2.25, 'median', 'or'),
 (2.25, 'max', 'and'),
 (2.25, 'max', 'or'),
 (2.5, 'median', 'and'),
 (2.5, 'median', 'or'),
 (2.5, 'max', 'and'),
 (2.5, 'max', 'or'),
 (2.75, 'median', 'and'),
 (2.75, 'median', 'or'),
 (2.75, 'max', 'and'),
 (2.75, 'max', 'or'),
 (3.0, 'median', 'and'),
 (3.0, 'median', 'or'),
 (3.0, 'max

In [31]:
len(all_combinations)

64

### AEDDM Framwework

In [32]:
def detect_at_batch_level(test_batch,b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list):
    
    # Layer 1 Variables 
    mse_list_layer1=[]     # Holds the recon loss values predicted by Layer 1 Autoencoder for a single batch
    exceed_count_layer1=0  # How many instances exceed layer one instance level threshold
    mse_sum=0              # sum of recon.error values from layer 1 AE for this batch
    
    
    
    # Layer 2 Variables 
    mse_list_layer2=[]      # Holds the recon.error values predicted by Layer 2 Autoencoder for a single batch
    exceed_count_layer2=-1  # If a batch is not passed to the layer2 AE , then exceed count value will be -1 . This is symbolic.
    mse_sum_layer2=0        # sum of recon.error values from layer 2 AE for this batch
    
        
    layer1_excede_list=[] # Holds the batch numbers of bathces exceeding layer1 thresholds
    #all_excede_list=[]    # Hold the batch number of batches exceeding both Layer-1 and Layer-2 AE Thresholds 
    layer_one_instance_exceed_list=[] # Holds the indices of instances exceeding layer 1 instance threshold
    layer_two_instance_exceed_list=[] # Holds the indices of instances exceeding layer 2 instance thresholds
     
      
    # Determine Layer 1 and Layer AE and their associated thresholds
    layer_one_batch_thres= batch_thres_pos if batch_thres_pos<batch_thres_neg  else batch_thres_neg
    layer_two_batch_thres= batch_thres_pos if batch_thres_pos>batch_thres_neg  else batch_thres_neg
    layer_one_encoder= encoder_pos_class if batch_thres_pos<batch_thres_neg else encoder_neg_class
    layer_two_encoder= encoder_pos_class if batch_thres_pos>batch_thres_neg else encoder_neg_class
    
    layer_one_count_threshold=count_thresh_pos if batch_thres_pos<batch_thres_neg  else count_thres_neg
    layer_two_count_threshold=count_thresh_pos if batch_thres_pos>batch_thres_neg  else count_thres_neg
    
    layer1_ins_thresh=instance_thresh_pos if  batch_thres_pos<batch_thres_neg  else instance_thresh_neg
    layer2_ins_thresh=instance_thresh_pos if  batch_thres_pos>batch_thres_neg  else instance_thresh_neg
    
    # Pass each instance of a batch to Layer 1 AE. Compute Batch MSE and Number of Instances excceding Layer 1 Instance Thresh       
    avg_mse_layer1=0
    for i in range(0,test_batch.shape[0]):
        ROW = np.array([test_batch[i]])
        pred= layer_one_encoder.predict(ROW) 
        mse = np.mean(np.power(test_batch[i] - pred, 2))
        mse_list_layer1.append(mse)
        if mse>layer1_ins_thresh:
            exceed_count_layer1+=1
            layer_one_instance_exceed_list.append(i)
        mse_sum+=mse    
    avg_mse_layer1=(mse_sum)/len(test_batch)
    avg_mse_layer2=0 
    # Check if This batch exceeds both Layer 1 batch and count thresholds
    if ((avg_mse_layer1>layer_one_batch_thres) and ( exceed_count_layer1 >layer_one_count_threshold) ):
        layer1_excede_list.append(b)# Keep track of batches exceesding layer 1 thresholds
        exceed_count_layer2=0
        # Pass each instance of this batch to Layer 2 AE. Compute Batch MSE and Number of Instances excceding Layer 2 Instance Thresh
        for i in range(0,test_batch.shape[0]):
            ROW = np.array([test_batch[i]])
            pred= layer_two_encoder.predict(ROW) 
            mse = np.mean(np.power(test_batch[i] - pred, 2))
            mse_list_layer2.append(mse)
            if mse>layer2_ins_thresh:
                exceed_count_layer2+=1
                layer_two_instance_exceed_list.append(i)
            mse_sum_layer2+=mse    
        avg_mse_layer2=(mse_sum_layer2)/len(test_batch)
        
        if  (avg_mse_layer2 > layer_two_batch_thres) and (exceed_count_layer2>layer_two_count_threshold):
            all_excede_list.append(b)# Keep track of batches exceesding layer 2 thresholds
          
    return  all_excede_list,mse_list_layer1 , exceed_count_layer1 ,avg_mse_layer2,exceed_count_layer2,len(layer_two_instance_exceed_list),layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list,layer_two_instance_exceed_list,avg_mse_layer1 

In [33]:
def detect_stream_drift(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1




In [34]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=float('inf') # No drift in the data
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values


In [35]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [36]:
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=40):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['class']
batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
#drifted=drifted_data[0:1280]
batches_d=make_batches(drifted_data)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    
  
    
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.25))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d

In [37]:
results_df_combined,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 0.0035685597722438433 , Count Threshold: 8.0
Thresholds for Nagative AE
Batch Thresold: 0.0064027380374840044 , Count Threshold: 7.5
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 8, 11, 12, 13, 17, 18, 21, 25, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 16, 17, 18, 19, 20, 21, 24, 27, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 23



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 5, 6, 7, 8, 9, 10, 12, 13, 16, 17, 18, 20, 23, 24, 25, 26, 27, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 4, 14, 15, 16, 19, 21, 22, 23, 24, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresh


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 5, 7, 8, 10, 11, 13, 15, 20, 23, 24, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 6, 7, 9, 10, 12, 14, 17, 19, 20, 21, 22, 23, 24, 26, 27, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 18



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 6, 7, 9, 17, 19, 20, 22, 23, 25, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 5, 7, 10, 12, 13, 16, 20, 21, 25, 26, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 6, 7, 10, 11, 12, 13, 15, 16, 18, 19, 21, 24, 25, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 25, 27, 31]

Number of Data Points Exceeding Layer 2 Enc


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 10, 11, 14, 15, 16, 17, 18, 19, 20, 23, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 26



**************

Batch Number : 46

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 10, 13, 17, 18, 19, 20, 22, 23, 25, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 28



**************

Batch Number : 47

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 6, 9, 13, 15, 20, 22, 23, 25, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 1


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 8, 11, 12, 13, 17, 18, 21, 25, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 16, 17, 18, 19, 20, 21, 24, 27, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 23



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 5, 6, 7, 8, 9, 10, 12, 13, 16, 17, 18, 20, 23, 24, 25, 26, 27, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 4, 14, 15, 16, 19, 21, 22, 23, 24, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 5, 6, 7, 11, 12, 13, 14, 15, 18, 20, 21, 22, 24, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 11, 12, 13, 16, 17, 20, 22, 24, 25, 26, 27, 28, 31]

Number 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 6, 7, 9, 17, 19, 20, 22, 23, 25, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 5, 7, 10, 12, 13, 16, 20, 21, 25, 26, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 6, 7, 10, 11, 12, 13, 15, 16, 18, 19, 21, 24, 25, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 25, 27, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 22



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 4, 5, 6, 8, 10, 11, 13, 19, 22, 25, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 5, 6, 7, 9, 11, 12, 14, 16, 17, 18, 20, 21, 23, 25, 27, 28]

Number of Data Points Exceeding Layer

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4, 5, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 41

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 7, 9, 14, 18, 23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [7, 9, 14, 18, 23]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 42

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 14] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 14, 17, 19, 28, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 43

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



*********


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [21]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 12

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [13]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 13

Data Points Exceeding Layer 1 Encoder Instance Th


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 45

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 46

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 47

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 48

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 19

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 20

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 21

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 53

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 54

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 55

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 56

Data Points Exceeding Layer 1 Encoder Instance Threshold : 

In [38]:
results_df_combined

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,46,46,37,0,0,-129
1,0.50,median,or,49,49,37,0,0,-138
2,0.50,max,and,5,5,37,0,0,-6
3,0.50,max,or,7,7,37,0,0,-12
4,0.75,median,and,39,39,37,0,0,-108
...,...,...,...,...,...,...,...,...,...
59,4.00,max,or,0,0,37,0,0,9
60,4.25,median,and,4,4,37,0,0,-3
61,4.25,median,or,5,5,37,0,0,-6
62,4.25,max,and,0,0,37,0,0,9


In [39]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

### Grid search uisng only one threshold

In [40]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift2(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=float('inf') # No drift in the data
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if batch_mse[i] > Thresh:
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values


In [41]:
def detect_stream_drift2(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift2(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1



In [42]:
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=40):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['class']
batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
#drifted=drifted_data[0:1280]
batches_d=make_batches(drifted_data)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    
  
    
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.25))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift2(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d

In [43]:
results_df_single,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 0.0035685597722438433 , Count Threshold: 8.0
Thresholds for Nagative AE
Batch Thresold: 0.0064027380374840044 , Count Threshold: 7.5
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 8, 11, 12, 13, 17, 18, 21, 25, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 16, 17, 18, 19, 20, 21, 24, 27, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 23



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 5, 6, 7, 8, 9, 10, 12, 13, 16, 17, 18, 20, 23, 24, 25, 26, 27, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 4, 14, 15, 16, 19, 21, 22, 23, 24, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresh


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 5, 7, 8, 10, 11, 13, 15, 20, 23, 24, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 6, 7, 9, 10, 12, 14, 17, 19, 20, 21, 22, 23, 24, 26, 27, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 18



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 6, 7, 9, 17, 19, 20, 22, 23, 25, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 5, 7, 10, 12, 13, 16, 20, 21, 25, 26, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 6, 7, 10, 11, 12, 13, 15, 16, 18, 19, 21, 24, 25, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 25, 27, 31]

Number of Data Points Exceeding Layer 2 Enc


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 10, 11, 14, 15, 16, 17, 18, 19, 20, 23, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 26



**************

Batch Number : 46

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 10, 13, 17, 18, 19, 20, 22, 23, 25, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 28



**************

Batch Number : 47

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 6, 9, 13, 15, 20, 22, 23, 25, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 1


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 8, 11, 12, 13, 17, 18, 21, 25, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 15, 16, 17, 18, 19, 20, 21, 24, 27, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 23



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 5, 6, 7, 8, 9, 10, 12, 13, 16, 17, 18, 20, 23, 24, 25, 26, 27, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 3, 4, 14, 15, 16, 19, 21, 22, 23, 24, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 5, 6, 7, 11, 12, 13, 14, 15, 18, 20, 21, 22, 24, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 11, 12, 13, 16, 17, 20, 22, 24, 25, 26, 27, 28, 31]

Number 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 6, 7, 9, 17, 19, 20, 22, 23, 25, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 5, 7, 10, 12, 13, 16, 20, 21, 25, 26, 28, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 6, 7, 10, 11, 12, 13, 15, 16, 18, 19, 21, 24, 25, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 25, 27, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 22



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 4, 5, 6, 8, 10, 11, 13, 19, 22, 25, 27, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 2, 4, 5, 6, 7, 9, 11, 12, 14, 16, 17, 18, 20, 21, 23, 25, 27, 28]

Number of Data Points Exceeding Layer

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4, 5, 28, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 4



**************

Batch Number : 41

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 7, 9, 14, 18, 23] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [7, 9, 14, 18, 23]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 5



**************

Batch Number : 42

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 14] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [3, 14, 17, 19, 28, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 6



**************

Batch Number : 43

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



*********


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [21] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [21]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 12

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 5] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [13]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 13

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 14

Data Points Exceeding Layer 1 Encoder Instance Th


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 46

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 47

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 48

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 49

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 19

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 20

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 21

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 54

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 55

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 56

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0

 Drift Detection at Batch Level

 Threshold exceeds at batch : 41
[41]
Warning Level at Batch 41

In [44]:
results_df_single

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,46,46,37,0,0,-129
1,0.50,median,or,46,46,37,0,0,-129
2,0.50,max,and,7,7,37,0,0,-12
3,0.50,max,or,7,7,37,0,0,-12
4,0.75,median,and,39,39,37,0,0,-108
...,...,...,...,...,...,...,...,...,...
59,4.00,max,or,0,0,37,0,0,9
60,4.25,median,and,4,4,37,0,0,-3
61,4.25,median,or,4,4,37,0,0,-3
62,4.25,max,and,0,0,37,0,0,9


In [45]:
results_df_combined

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,46,46,37,0,0,-129
1,0.50,median,or,49,49,37,0,0,-138
2,0.50,max,and,5,5,37,0,0,-6
3,0.50,max,or,7,7,37,0,0,-12
4,0.75,median,and,39,39,37,0,0,-108
...,...,...,...,...,...,...,...,...,...
59,4.00,max,or,0,0,37,0,0,9
60,4.25,median,and,4,4,37,0,0,-3
61,4.25,median,or,5,5,37,0,0,-6
62,4.25,max,and,0,0,37,0,0,9


In [46]:
pd.options.display.max_rows = 100 
results_df_combined

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,46,46,37,0,0,-129
1,0.50,median,or,49,49,37,0,0,-138
2,0.50,max,and,5,5,37,0,0,-6
3,0.50,max,or,7,7,37,0,0,-12
4,0.75,median,and,39,39,37,0,0,-108
5,0.75,median,or,41,41,37,0,0,-114
6,0.75,max,and,4,4,37,0,0,-3
7,0.75,max,or,6,6,37,0,0,-9
8,1.00,median,and,28,28,37,0,0,-75
9,1.00,median,or,30,30,37,0,0,-81


In [47]:
results_df_single

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,46,46,37,0,0,-129
1,0.50,median,or,46,46,37,0,0,-129
2,0.50,max,and,7,7,37,0,0,-12
3,0.50,max,or,7,7,37,0,0,-12
4,0.75,median,and,39,39,37,0,0,-108
5,0.75,median,or,39,39,37,0,0,-108
6,0.75,max,and,6,6,37,0,0,-9
7,0.75,max,or,6,6,37,0,0,-9
8,1.00,median,and,28,28,37,0,0,-75
9,1.00,median,or,28,28,37,0,0,-75


In [48]:
results_df_combined['WnDs'].mean()

8.234375

In [49]:
results_df_single['WnDs'].mean()

8.1875

### Piared two tail t-test

In [50]:
df1=results_df_combined[results_df_combined.index % 2 == 0]
df2=results_df_single[results_df_single.index % 2 == 0]

### two-tailed Test

In [51]:
from scipy.stats import ttest_rel

t_statistic, p_value = ttest_rel(df1['WnDs'], df2['WnDs'])

# Print the results
print(f"t-statistic: {t_statistic:.3f}")
print(f"p-value: {p_value:.3f}")

# Interpret the results
alpha = 0.10  # Significance level
if p_value > alpha:
    print("The null hypothesis cannot be rejected.")
else:
    print("The null hypothesis can be rejected.")

t-statistic: -2.463
p-value: 0.020
The null hypothesis can be rejected.


### one-tailed Test

In [52]:
# Perform the one-tailed paired t-test (right-tailed)
t_statistic, p_value = ttest_rel(df2['WnDs'], df1['WnDs'], alternative='greater') 
# Print the results
print(f"t-statistic: {t_statistic:.3f}")
print(f"p-value: {p_value:.3f}")

# **Null Hypothesis (H0):** The ablated detector produces less than or equal to the number of warnings as the full detector.
# **Alternative Hypothesis (H1):** The ablated detector produces significantly more warnings than the full detector.

# Interpret the results
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis. The ablated detector produces significantly more warnings than the full detector.")
else:
    print("Fail to reject the null hypothesis.") 

t-statistic: 2.463
p-value: 0.010
Reject the null hypothesis. The ablated detector produces significantly more warnings than the full detector.


In [54]:
df1['WnDs'].mean()

7.46875

In [55]:
df1

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,46,46,37,0,0,-129
2,0.50,max,and,5,5,37,0,0,-6
4,0.75,median,and,39,39,37,0,0,-108
6,0.75,max,and,4,4,37,0,0,-3
8,1.00,median,and,28,28,37,0,0,-75
10,1.00,max,and,0,0,37,0,0,9
12,1.25,median,and,23,23,37,0,0,-60
14,1.25,max,and,0,0,37,0,0,9
16,1.50,median,and,20,20,37,0,0,-51
18,1.50,max,and,0,0,37,0,0,9


In [56]:
df2

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,46,46,37,0,0,-129
2,0.50,max,and,7,7,37,0,0,-12
4,0.75,median,and,39,39,37,0,0,-108
6,0.75,max,and,6,6,37,0,0,-9
8,1.00,median,and,28,28,37,0,0,-75
10,1.00,max,and,6,6,37,0,0,-9
12,1.25,median,and,23,23,37,0,0,-60
14,1.25,max,and,6,6,37,0,0,-9
16,1.50,median,and,20,20,37,0,0,-51
18,1.50,max,and,4,4,37,0,0,-3
